In [1]:
import pandas as pd
import numpy as np
import pickle
import random
import os
import math
import sim_functions_cleaner  as sf
import operator
import study
import time

In [2]:
##for each person generate dates
##easiest to take from original data, this might best mimic actual situation 

In [3]:
#root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/processed/'

In [4]:
experiment = study.study()

In [5]:
experiment.population[1]

In [6]:
#days = pd.date_range(start = '7/22/2015',end =pd.Timestamp('7/22/2015')+pd.DateOffset(days=45),freq='30T')

In [7]:
#ft = experiment.study_days[96]

In [8]:
#yesterday = ft-pd.DateOffset(days=1)
#end = yesterday.replace(hour=0,minute=0)
#start = experiment.study_days[0]
#end

In [11]:
def new_kind_of_simulation(experiment,policy=None):
    

    for time in experiment.study_days:
            
            
            
            
            ##update global context
            ##global context shared across all participants
            tod = sf.get_time_of_day(time)
            dow = sf.get_day_of_week(time)
            if time==experiment.study_days[0]:
                weather = sf.get_weather_prior(tod,time.month)
            elif time.hour in experiment.weather_update_hours and time.minute==0:
                weather = sf.get_next_weather(str(tod),str(time.month),weather)
            ##location depends on person 
            
            for person in experiment.dates_to_people[time]:
                action = -1 
                #1
                ##for every active person update person specific aspects of their context
                participant = experiment.population[person]
                #update global context variables
                participant.set_tod(tod)
                participant.set_dow(dow)
                participant.set_wea(weather)
                
                if time == participant.times[0]:
                    #get first location 
                    location = sf.get_location_prior(str(participant.gid),str(dow),str(tod))
                    
                    
                if time <= participant.times[1]:
                    steps_last_time_period = 0  
                    
                    ##set first pre-treatment, yesterday step count, variation and dosage
                else:
                    #print(time)
                    steps_last_time_period = participant.find_last_time_period_steps(time)
                    
             
                    #get var id
                    
                if time.date() <= participant.times[0].date():
                    steps_yesterday = 0    
                else:
                    steps_yesterday =  participant.find_yesterday_steps(time)
                    steps_yesterday = sf.to_yid(steps_yesterday)
                    
                if time.hour in experiment.location_update_hours:
                    location = sf.get_next_location(participant.gid,dow,tod,participant.get_loc())
                
                if time.date()>(participant.times[0]+pd.DateOffset(days=1)).date():
                  
                    if time.hour==0:
                        variation = participant.find_variation(time)
                else:
                    variation = 1 
                
                participant.set_loc(location)
                ##maybe faster to update instead of query?
                participant.set_last_time_period_steps(steps_last_time_period)
                participant.set_yesterday_steps(steps_yesterday)
                participant.set_variation(variation)
                
                ##continue
                #2
                ##for every active person take an action according to current context, policy, and parameters
                
                
                ##for now:
                if time in participant.decision_times:
                    action = sf.get_action(policy)
                    steps = 100
                    #steps = sf.get_steps()
                else:
                    steps = sf.get_steps_no_action(participant.gid,tod,dow,location,weather)
                
                
                
                #steps = 100 
                
                ##history:
                context_dict = {'steps':steps,'action':0,'weather':weather,'location':location,\
                                'ltps':steps_last_time_period}
                participant.history[time]=context_dict
            
            #3
            ##for every active person generate a step count given current context
            
            
            
            
            ##update at midnight (here we have ensured that no one has a ) 
            


            




In [12]:
start = time.time()
new_kind_of_simulation(experiment)
end = time.time()
print(end-start)

147.26273608207703


In [13]:
#set([w['steps'] for w in experiment.population[32].history.values()])

{0.0,
 1.519986408990036,
 2.717386066235065,
 4.0588151177709335,
 5.626168051395467,
 6.158066193636188,
 6.247873195842033,
 6.544877456696213,
 6.87357306672547,
 9.145842590146515,
 11.566854076262047,
 12.791406181650984,
 12.92923375840033,
 13.034752441454117,
 13.773389593920996,
 16.337394344597243,
 16.820851322672723,
 16.84347804325205,
 16.920376799030578,
 17.083869688527443,
 17.638834640633604,
 17.851835873222655,
 18.752729503496937,
 19.4298777178077,
 20.977594068945024,
 23.543756679608776,
 23.719933027787533,
 23.894020586737724,
 25.66100474107074,
 25.754309918826294,
 26.703944978483065,
 27.432077808530636,
 28.198542492825766,
 28.20607258280353,
 28.38484076776942,
 29.236987058462148,
 31.90771420896607,
 32.02060392886679,
 32.14683696424089,
 32.8542701560491,
 33.00601372539603,
 34.87587455444529,
 34.91908509363256,
 35.32845338798609,
 35.983871947327884,
 37.49315737414389,
 38.64576474417672,
 39.4657753024257,
 40.75339344999952,
 41.604993266466

In [5]:
def get_today_variance(today,lookup,all_steps):
    morning = today.replace(hour=0,minute=0)
    
    tonight = today.replace(hour=23,minute=30)
    
    steps = all_steps[lookup[morning]:lookup[tonight]+1]
    
   
    
    steps = transform_to_hour(steps)
    
    return np.array(steps).std()

In [6]:
means = []
for n in range(1000):
    gr=sf.simulate_run(1,days,[])
    means.append(np.array(gr[0][0]).mean())

In [49]:
np.array(gr[0][0]).mean()

446.51003316358634

In [7]:
np.array(means).mean()

137.29867338336618

In [38]:
gr[1][0]

[2, 0, 0, 0, 2, 1, 0, 1, 1]

In [13]:
get_today_variance(gr[0],gr[1],gr[2])

False


0.0

In [14]:
def get_all_day_slices(first,last):
    slices = []
    day = first
    
    while day<last:
        slice_one = [day]
        #slice_one.append(day.replace(hour=23,minute=30))
        
        day_two = day+pd.DateOffset(days=1)
        slice_one.append(day_two)
        slices.append(slice_one)
        day=day_two
    return slices

In [8]:
def transform_to_hour(steps):
    to_return = []
    for i in range(0,len(steps)-1,2):
        to_return.append(steps[i]+steps[i+1])
    return to_return

In [35]:
def day_slices_to_median(day_slices,day_lookup,all_steps):
    days_steps = []
    for ds in day_slices:
        steps = all_steps[day_lookup[ds[0]]:day_lookup[ds[1]]]
       
        steps = transform_to_hour(steps)
        print(len(steps))
        days_steps.append(np.array(steps).std())

    return np.median(np.array(days_steps))
        

In [19]:
x=get_all_day_slices(gr[3],gr[4])

In [43]:
gr[4]

Timestamp('2015-07-29 00:00:00', freq='30T')

In [38]:
day_slices_to_median(x,gr[2],gr[5])

24
24
24
24
24
24
24


0.0

In [3]:
    last_index = dd[one_week_ago.date()][0]
    
    day_slices=get_all_day_slices(last_index,dd[i.date()][0])
    
    median = day_slices_to_median(day_slices,gr[2],gr[5])

You can pass in a bunch of decision times. I just tested with one. 

Right now I think the number of people is being totally ignored. This needs to be fixed. 

In [12]:
x=sim_functions.simulate_run(num_people,temp_indices,decision_times)

NameError: name 'temp_indices' is not defined

In [14]:
np.array(x).mean()
#np.array(x).std()

207.01370855122792

In [15]:
np.array(x).mean()
np.array(x).std()

267.8619020654599

In [ ]:
2-2-0-1-1-0-0-1

In [5]:
root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/temporary/'
data_root =  '../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/'


In [9]:
with open('../../../../Volumes/dav/HeartSteps/pooling_rl_shared_data/distributions/steps_both_groups_logs_dosage_estf.pkl'.format(),'rb') as f:
    df = pickle.load(f)

In [10]:
df.keys()

dict_keys(['2-0-2-1-2-1-0-3-18', '2-1-1-1-0-1-0-3-77', '2-0-0-0-3-3-0-3-9', '2-0-2-0-2-2-0-4-37', '2-1-2-1-0-1-0-4-70', '2-0-1-0-2-0-0-4-35', '2-1-2-1-0-1-0-3-13', '2-0-2-2-2-0-0-4-48', '2-0-1-1-2-1-0-3-28', '2-0-0-2-1-1-0-3-8', '2-0-2-0-2-0-0-3', '2-1-1-1-0-2-0-4-78', '2-0-1-1-2-1-0-3-31', '2-0-1-1-1-1-0-4-70', '2-0-1-2-1-3-0-4-66', '2-0-2-1-0-1-0-3-31', '2-0-1-2-0--1-0-4-25', '1-0-2-0-3-0-0-4-16', '2-1-2-0-3-2-0-4-3', '2-0-0-1-0-1-0-3', '2-1-1-1-1-1-0-3-18', '2-1-1-0-1-1-0-4-53', '2-1-2-2-1-1-0-4-35', '2-0-1-2-0--1-0-3-31', '2-0-1-2-0-3-0-4-64', '2-0-2-1-1-2-0', '1-1-1-0-3-3-0-3-38', '2-0-0-1-2-1-0-4-77', '2-0-2-2-1-1-1', '2-0-1-2-1-2-0', '2-0-2-2-0-2-1-3-3', '2-0-1-1-1-1-1-3-9', '1-0-2-0-0-0-0-4-94', '2-0-2-1-1-2-0-3-22', '2-0-1-2-0-0-1-4-9', '2-0-2-0-2-2-0-4-39', '2-1-1-1-2-0-0-4-91', '2-0-2-1-0-2-0-3-21', '2-0-1-0-0-0-0-4-38', '2-0-2-1-0-3-0-3-96', '2-0-2-0-0-1-1-4-1', '2-0-1-1-2-0-0-3-47', '2-0-1-1-1-2-0-4-9', '2-0-1-0-1-1-0-4-30', '2-0-1-1-0-2-0-3-53', '2-1-1-1-0-0-0-4-95', '2-1